# Youtrack. Time in status statistics

## Description
A Jupyter Notebook that helps understand how long each YouTrack issue stays in each status. It calculates various metrics and statistics.

### Instructions
1. All variables set in UPPER_CASE can be and should be changed.
2. Execute the cells from top to bottom from "Build DataFrame" section
3. After you can execute the remaining sections in any order, but ensure that all UPPER_CASE variables are filled.
4. Once the DataFrame is built, you can call `get_issue_info()` for any issue to retrieve detailed information about it.

In [ ]:
# Constants for project identification and data filtering

PROJECT_ID = ""         # Project name
STATUS_FIELD = "State"  # Field that indicates the status of the work
TYPE_FIELD = "Type"     # Field that indicates the type of the work

# Filter for the project issues
FILTER = f"project: {PROJECT_ID} created: 2025-02-01 .. 2025-12-31"

## Build DataFrame

In [ ]:
# library import

import requests
from datetime import datetime, timedelta
import pandas as pd

import activities
import issues

from helpers import format_time, create_column_mapping

In [ ]:
# Get all issues in specified project by filter

issue_list = issues.get_all_issues(PROJECT_ID, FILTER)
print(f"Total issues received from project {PROJECT_ID}: {len(issue_list)}")

In [ ]:
# Get status change events for each issue
for issue in issue_list:
    issue_id = issue['idReadable']
    if 'activities' not in issue:
        activity_list = activities.filter_and_sort_activities(activities.get_issue_activities(issue_id), STATUS_FIELD)
        issue['activities'] = activity_list

In [ ]:
# Create DataFrame for analysis
data = issues.create_dataframe(issue_list)
df = pd.DataFrame(data)

In [ ]:
# Additional analytics on issues

# Calculate queue time
QUEUE_STATE = ["New", "Queue"]

queue_columns = [f'{state} duration' for state in QUEUE_STATE]
df['Queue time'] = df[queue_columns].sum(axis=1)

# Calculate Lead Time and Cycle Time ror all resolved issues
filtered_df = df[df['resolved'].notna()] 
df.loc[filtered_df.index, 'Lead time'] = filtered_df['resolved'] - filtered_df['created']
df.loc[filtered_df.index, 'Cycle time'] = df.loc[filtered_df.index, 'Lead time'] - df.loc[filtered_df.index, 'Queue time']

# Helper variables and functions for pretty analytics
column_mapping = create_column_mapping(df.columns)

def get_issue_info(issue_id):
    print(df.loc[df['issue_id'] == issue_id, ['issue_id', TYPE_FIELD, STATUS_FIELD, 'Priority']].rename(columns={'issue_id': 'ID'}), "\n")
    time_columns = df.filter(regex=r'duration$|Spent time')             # Filert columns
    result = time_columns.loc[df['issue_id'] == issue_id].dropna(axis=1) # Select row by specific id
    result = result.map(format_time).rename(columns=column_mapping)
    return result

## DataFrame

In [ ]:
# DataFrame
df

In [ ]:
# To display a list of unique values in a field, you can use the following commands

#print("State: ", df['State'].unique())
#print("Priority: ", df['Priority'].unique())
#print("Type: ", df['Type'].unique())

## Аналитика

In [ ]:
# General table for average time in status, grouped by issue type

time_columns = [col for col in df.columns if col.endswith('duration')]
average_time_by_content_type = df.groupby(TYPE_FIELD)[time_columns].mean()
average_time_by_content_type.map(format_time).rename(columns=column_mapping)

In [ ]:
# Filtering issue types that require analytics
# Grouping criteria

ISSUE_TYPE = ["Bug"]         # Can specify multiple types
GROUPING = "Priority"    # Can specify only one field

In [ ]:
# Calculate the average time an issue spends in each status

print(f"Statistics for {', '.join(ISSUE_TYPE)}")

time_columns = [col for col in df.columns if col.endswith('duration')]
average_time_by_content_type = df[df[TYPE_FIELD].isin(ISSUE_TYPE)].groupby(GROUPING)[time_columns].mean()
average_time_by_content_type.map(format_time).rename(columns=column_mapping)

In [ ]:
# Average time an issue spends in each status without grouping

filtered_df = df[df['Type'].isin(ISSUE_TYPE)]
filtered_df.filter(regex=r'duration$').rename(columns=column_mapping).mean().dropna().apply(format_time)

In [ ]:
# Avarage metrics

columns = ['Queue time', 'Cycle time', 'Lead time', 'Spent time']

average_time_to_resolve = df.loc[filtered_df.index, 'Lead time'].mean()
average_time_in_work = df.loc[filtered_df.index, 'Cycle time'].mean()
average_time_in_queue = df.loc[filtered_df.index, 'Queue time'].mean()
grouped_means = filtered_df.groupby(GROUPING)[columns].mean()

print(f"Queue time: {format_time(average_time_in_queue)}")
print(f"Cycle time: {format_time(average_time_in_work)}")
print(f"Lead time: {format_time(average_time_to_resolve)}\n")

grouped_means.map(format_time).rename(columns={f"{QUEUE_STATE} duration":"Queue"})

## Anomalies

In [ ]:
# Filtering issues types that require analytics
# Grouping criteria

ISSUE_TYPE = ["Bug"]         # Can specify multiple types
GROUPING = "Priority"   # Can specify only one field

In [ ]:
# Max lead time issue

df_filtered_2 = df[df[TYPE_FIELD].isin(ISSUE_TYPE)].dropna(subset=['Lead time'])
max_time_to_resolve = df_filtered_2.groupby(GROUPING)['Lead time'].idxmax()
max_resolve_issue = df_filtered_2.loc[max_time_to_resolve, [GROUPING, 'issue_id', 'Lead time', 'Cycle time', 'Spent time']]
max_resolve_issue['Lead time'] = max_resolve_issue['Lead time'].apply(format_time)
max_resolve_issue['Cycle time'] = max_resolve_issue['Cycle time'].apply(format_time)
max_resolve_issue['Spent time'] = max_resolve_issue['Spent time'].apply(format_time)

max_resolve_issue

In [ ]:
# Using get_issue_info anywhere in the notebook, you can get data for a specific issue
get_issue_info("")

In [ ]:
# Max time in queue

df_filtered_3 = df[df[TYPE_FIELD].isin(ISSUE_TYPE)].dropna(subset=['Queue time'])
max_time_in_queue = df_filtered_3.groupby(GROUPING)['Queue time'].idxmax()
max_queue_issue = df_filtered_3.loc[max_time_in_queue, [GROUPING, 'issue_id', 'Type', 'Queue time']]
max_queue_issue['Queue time'] = max_queue_issue['Queue time'].apply(format_time)

max_queue_issue.rename(columns=column_mapping)